In [1]:
import os
os.environ["PYOPENGL_PLATFORM"] = "egl" #opengl seems to only work with TPU

from absl import flags, app
import sys
sys.path.insert(0,'third_party')
import numpy as np
import glob
import pdb
import cv2
from scipy.spatial.transform import Rotation as R

from nnutils.train_utils import v2s_trainer

from explore import get_center_crop_img

opts = flags.FLAGS

Jitting Chamfer 3D
Loaded JIT 3D CUDA chamfer distance


In [23]:
# logdir='logdir/dog80-v0/'
# seqname='dog80'
# logdir='logdir/human47/'
# seqname='human47'
logdir='logdir/cat76/'
seqname='cat76'
# params
img_size = 512
grid_size= 128

show_rest_pose = False
show_deform = False
apply_lbs = True
mlp_deform = False


cam_offset = np.asarray([0,0,0.6])

focal_fac = 2

# load flags
args = opts.read_flags_from_files(['--flagfile=%s/opts.log'%logdir, 
                                   '--nolineloade',
                                   'seqname=%s'%seqname, 
                                   '--full_mesh', 
                                   '--noce_color', 
                                   ])
unknown_flags, unparsed_args = opts._parse_args(args, known_only=True)
opts.mark_as_parsed()
opts.validate_all_flags()

from IPython.utils import io
with io.capture_output() as captured:
    if opts.logname not in opts.model_path:
        opts.model_path = 'logdir/%s/params_latest.pth'%opts.logname
    opts.bgmlp=''
    opts.bg_path=''
    #opts.lineload=False
    #opts.mlp_deform=False
    trainer = v2s_trainer(opts, is_eval=True)
    data_info = trainer.init_dataset()
    trainer.define_model(data_info)
    seqname=opts.seqname

    model = trainer.model
    model.eval()

In [24]:
# merge all jpegs into videos
import einops
import mediapy
rnum=8 # per row
cnum=8 # per col
tnum=50 # 5s

videos = sorted(glob.glob('%s/*}.mp4'%logdir), key=lambda x: int(x.split('{')[-1].split('}')[0]))
#videos = [name for i,name in enumerate(videos) if i!=35]
#videos = videos[::-1]
if len(videos)<rnum*cnum:
    videos = videos + videos[:rnum*cnum-len(videos)]

frames_all = []
for vidname in videos[:rnum*cnum]:
    vidname_raw = vidname.replace('}.mp4', '}-vid.mp4')
    vidname = vidname.replace('}.mp4', '}.mp4')
    frames_raw=[]
    frames=[]
    print(vidname_raw)
    with mediapy.VideoReader(vidname_raw) as reader:
        for frame in reader:
            frames_raw.append(frame)
        
    with mediapy.VideoReader(vidname) as reader:
        idx=0
        for img in reader:
            # read image
            frame = frames_raw[idx]
            sil = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)<255
            img = get_center_crop_img(img, sil, img_size=img_size//4)
            frame = get_center_crop_img(frame, sil, img_size=img_size//4)
            frame = np.concatenate([frame, img], axis=1)
            frames.append(frame/255)
            idx+=1
        if idx<tnum:
            for _ in range(tnum-idx):
                frames.append(frame/255)
        if idx>tnum:
            frames = frames[:tnum]
    frames_all.append(frames)
frames_all =np.concatenate(frames_all, axis=0)
frames_all = einops.rearrange(frames_all, '(v1 v2 t) h w c -> t (v1 h) (v2 w) c', v1=rnum, v2=cnum)
mediapy.show_video(frames_all, fps=10)

logdir/cat76/cat76-{0}-vid.mp4


logdir/cat76/cat76-{1}-vid.mp4
logdir/cat76/cat76-{2}-vid.mp4
logdir/cat76/cat76-{3}-vid.mp4
logdir/cat76/cat76-{4}-vid.mp4
logdir/cat76/cat76-{5}-vid.mp4
logdir/cat76/cat76-{6}-vid.mp4
logdir/cat76/cat76-{7}-vid.mp4
logdir/cat76/cat76-{8}-vid.mp4
logdir/cat76/cat76-{9}-vid.mp4
logdir/cat76/cat76-{10}-vid.mp4
logdir/cat76/cat76-{11}-vid.mp4
logdir/cat76/cat76-{12}-vid.mp4
logdir/cat76/cat76-{13}-vid.mp4
logdir/cat76/cat76-{14}-vid.mp4
logdir/cat76/cat76-{15}-vid.mp4
logdir/cat76/cat76-{16}-vid.mp4
logdir/cat76/cat76-{17}-vid.mp4
logdir/cat76/cat76-{18}-vid.mp4
logdir/cat76/cat76-{19}-vid.mp4
logdir/cat76/cat76-{20}-vid.mp4
logdir/cat76/cat76-{21}-vid.mp4
logdir/cat76/cat76-{22}-vid.mp4
logdir/cat76/cat76-{23}-vid.mp4
logdir/cat76/cat76-{24}-vid.mp4
logdir/cat76/cat76-{25}-vid.mp4
logdir/cat76/cat76-{26}-vid.mp4
logdir/cat76/cat76-{27}-vid.mp4
logdir/cat76/cat76-{28}-vid.mp4
logdir/cat76/cat76-{29}-vid.mp4
logdir/cat76/cat76-{30}-vid.mp4
logdir/cat76/cat76-{31}-vid.mp4
logdir/cat76/cat7

This browser does not support the video tag.


In [17]:
# save to tmp/$seqname.mp4
mediapy.write_video('tmp/%s.mp4'%seqname, frames_all, fps=10)